<a href="https://colab.research.google.com/github/Veena9977/Practice2/blob/main/Deeplearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import tensorflow as tf
from tensorflow.keras.layers import SimpleRNN, LSTM, Dense, Dropout, Embedding, BatchNormalization, GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from sklearn.model_selection import train_test_split

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
data = pd.read_csv('/content/judge-1377884607_tweet_product_company.csv', encoding='MacRoman')

In [ ]:
#Display the first five rows
data.head()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion


In [ ]:
#Dropt he table named 'emotion_in_tweet_is_directed_at'
data.drop('emotion_in_tweet_is_directed_at', axis=1, inplace=True)


In [ ]:
data.head()

,tweet_text,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Positive emotion


In [ ]:
#Check the dtypes
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9093 entries, 0 to 9092
Data columns (total 2 columns):
 #   Column                                              Non-Null Count  Dtype 
---  ------                                              --------------  ----- 
 0   tweet_text                                          9092 non-null   object
 1   is_there_an_emotion_directed_at_a_brand_or_product  9093 non-null   object
dtypes: object(2)
memory usage: 142.2+ KB
None


In [ ]:
#Checking missing values
data.isna().sum()

,0
tweet_text,1
is_there_an_emotion_directed_at_a_brand_or_product,0


In [ ]:
#Drop the missing value
data = data.dropna()

In [ ]:
#Checking missing values
data.isna().sum()

,0
tweet_text,0
is_there_an_emotion_directed_at_a_brand_or_product,0


In [ ]:
#Convert object type to string type
data['tweet_text'] = data['tweet_text'].astype(str)

In [ ]:
#Define stop word
stop_words = set(stopwords.words('english'))


In [ ]:
#Remove stop words,convert to lower case and tokenize the words
def clean_reviews(text):
    if isinstance(text, str):  # Check if the text is a string
        text = text.lower()  # Convert to lower case
        text = word_tokenize(text)  # Tokenization of words
        text = [word for word in text if word not in stop_words]  # Stop words removal
    else:
        text = []  # Handle non-string or NaN values as empty list
    return text

In [ ]:
#Convert to string type
data['tweet_text'] = data['tweet_text'].astype(str)

In [ ]:
#Fill nan values with empty string
data['tweet_text'] = data['tweet_text'].fillna('')

In [ ]:
#Apply the clean_review function
data['cleaned_tweet'] = data['tweet_text'].apply(clean_reviews)


In [ ]:
#Define x and y .Here y is the target variable.
X = data['tweet_text']
y = data['sentiment_encoded']

In [ ]:
#Tokenize  and convert to sequence list
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)

In [ ]:
X_seq = tokenizer.texts_to_sequences(X)

In [ ]:
#Convert to sequence list
max_words = 500
X_pad = pad_sequences(X_seq, maxlen=max_words)

In [ ]:
#Encode the target variables into integer values
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
y_encoded = encoder.fit_transform(data['is_there_an_emotion_directed_at_a_brand_or_product'])

In [ ]:
#Convert to train and test split
X_train, X_test, y_train, y_test = train_test_split(X_pad, y_encoded, test_size=0.2, random_state=42)

In [ ]:
from keras.optimizers import Adam

LSTM MODEL

In [ ]:
model = Sequential(name="LSTM_Model")
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128, input_length=500))

# adding a LSTM layer
model.add(LSTM(256, return_sequences=False))
BatchNormalization()
model.add(Dropout(0.5))

#adding a dense layer with activation function of relu
model.add(Dense(256, activation='relu'))
BatchNormalization()
model.add(Dropout(0.5))

# adding the final output activation with activation function of softmax
model.add(Dense(4, activation='softmax'))

# printing model summary
print(model.summary())

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "LSTM_Model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_6 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_6 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_12 (Dropout)                 │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_13 (Dropout)                 │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [ ]:
#Compile the model
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=Adam(learning_rate=0.001),
              metrics=['accuracy'])

In [136]:
#Train the model
history = model.fit(X_train, y_train,
                    batch_size=64,
                    epochs=5,
                    verbose=1,
                    validation_data=(X_test, y_test))

Epoch 1/5
114/114 ━━━━━━━━━━━━━━━━━━━━ 290s 3s/step - accuracy: 0.7500 - loss: 0.6311 - val_accuracy: 0.6465 - val_loss: 0.8811
Epoch 2/5
114/114 ━━━━━━━━━━━━━━━━━━━━ 342s 3s/step - accuracy: 0.8201 - loss: 0.4765 - val_accuracy: 0.6460 - val_loss: 0.9301
Epoch 3/5
114/114 ━━━━━━━━━━━━━━━━━━━━ 291s 3s/step - accuracy: 0.8608 - loss: 0.3565 - val_accuracy: 0.6504 - val_loss: 1.0473
Epoch 4/5
114/114 ━━━━━━━━━━━━━━━━━━━━ 319s 3s/step - accuracy: 0.8906 - loss: 0.2893 - val_accuracy: 0.6520 - val_loss: 1.1191
Epoch 5/5
114/114 ━━━━━━━━━━━━━━━━━━━━ 286s 3s/step - accuracy: 0.9022 - loss: 0.2568 - val_accuracy: 0.6564 - val_loss: 1.1527


In [137]:
loss, accuracy = model.evaluate(X_test_pad, y_test)

57/57 ━━━━━━━━━━━━━━━━━━━━ 27s 470ms/step - accuracy: 0.5231 - loss: 1.5906


In [138]:
#Display the accuracy
print(f"Test Accuracy: {accuracy}")

Test Accuracy: 0.5316107869148254
